In [2]:
%load_ext autoreload
%autoreload 2
import yaml
import pandas as pd
import logging
from matching_ilp_refactor import MatchingILP
from lp_solver import solve
import json
from importlib import reload
import compute_scores
import create_indicator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
logging.basicConfig(level=logging.INFO,format='%(asctime)s, %(levelname)s, %(message)s')
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
logger = logging.getLogger("NB")

In [4]:
reload(compute_scores)
from compute_scores import compute_scores

co_review_vars=None
with open('ilp_config_refactor.yml', 'rb') as fh:
    config = yaml.load(fh,Loader=yaml.FullLoader)
    
scores_df = compute_scores(config)
bids_df = pd.read_csv(config['BIDS_FILE']).set_index(['paper','reviewer'])

In [5]:
reviewer_df = pd.read_csv(config['REVIEWERS_FILE']).set_index('reviewer')
reviewer_df['conflict_papers'] = reviewer_df['conflict_papers'].apply(lambda x: json.loads(x))
reviewer_df['authored'] = reviewer_df['authored'].apply(lambda x: json.loads(x))

In [6]:
reviewers = reviewer_df.index.values
papers = scores_df.index.unique('paper').values
missing_reviewers = set(scores_df.index.unique('reviewer')) - set(reviewers)

In [7]:
scores_df = scores_df[~scores_df.index.get_level_values('reviewer').isin(list(missing_reviewers))]

In [8]:
reload(create_indicator)
from create_indicator import create_paper_reviewer_df
paper_reviewer_df = create_paper_reviewer_df(scores_df=scores_df, 
                reviewer_df=reviewer_df,
                bids_df=bids_df, 
                k=10,
                score_threshold=0.15)

/global/scratch/chrisc/projects/AAAI21_Matching/aaai21/create_indicator.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_df['role'] = scores_df.apply(lambda row: reviewer_df.loc[row.name[1],'role'],axis=1)
Adding best reviewers for papers: 100%|██████████| 8676/8676 [01:59<00:00, 72.50it/s]


In [9]:
paper_reviewer_df = paper_reviewer_df.rename(columns={'agg_score':'score'})

In [ ]:
distance_df = pd.read_csv(config['COAUTHOR_DISTANCE_FILE'])

In [16]:
import matching_ilp_refactor
reload(matching_ilp_refactor)
from matching_ilp_refactor import MatchingILP
ilp = MatchingILP(paper_reviewer_df,reviewer_df,distance_df,config,co_review_vars,basename='test',add_soft_constraints=True)
ilp.add_paper_distribution_constraints_obj_limits(role='SPC',num_papers_list=[8,12,16,20,24])

In [17]:
problem_path = ilp.create_ilp()

2021-09-24 13:39:30,258, INFO, Reviewer matching objective
2021-09-24 13:40:01,153, INFO, Paper capacity
Building paper capacity constraints...: 100%|██████████| 8617/8617 [03:01<00:00, 47.61it/s]
2021-09-24 13:43:02,171, INFO, Reviewer capacity
Building reviewer capacity constraints...: 9796it [00:39, 247.73it/s]
2021-09-24 13:43:41,722, INFO, Coreview Constraints
2021-09-24 13:43:41,724, WARNING, Co review var empty. Not adding any coreview constraints
2021-09-24 13:43:41,726, INFO, Co-author distance Objective
2021-09-24 13:43:41,728, WARNING, Co review var empty. Cannot contruct coauthor distance constraints
2021-09-24 13:43:41,730, INFO, Seniority Objectives
Building seniority constraints...: 100%|██████████| 8617/8617 [00:56<00:00, 152.38it/s]
2021-09-24 13:44:38,288, INFO, Region Objective
2021-09-24 13:44:38,298, INFO, Region Constraints and bounds
2021-09-24 13:45:52,125, INFO, Cycle Bounds
Building bidding cycles...: 9554it [06:55, 23.00it/s] 
2021-09-24 13:52:50,622, INFO, S

In [18]:
solution_file = solve(problem_path=problem_path)

Status 101: integer optimal solution
Objective 21893.346000016376
Wrote solution to test.sol
